<center><img src="car.jpg" width=500></center>


Insurance companies invest a lot of time and money into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

(`Source: https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf`) 

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [54]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# reading the file
data_insurance = pd.read_csv('car_insurance.csv')
data_insurance.head()

,id,age,gender,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents,outcome
0,569520,3,0,0-9y,high school,upper class,0.629027,1.0,after 2015,0.0,1.0,10238,12000.0,sedan,0,0,0,0.0
1,750365,0,1,0-9y,none,poverty,0.357757,0.0,before 2015,0.0,0.0,10238,16000.0,sedan,0,0,0,1.0
2,199901,0,0,0-9y,high school,working class,0.493146,1.0,before 2015,0.0,0.0,10238,11000.0,sedan,0,0,0,0.0
3,478866,0,1,0-9y,university,working class,0.206013,1.0,before 2015,0.0,1.0,32765,11000.0,sedan,0,0,0,0.0
4,731664,1,1,10-19y,none,working class,0.388366,1.0,before 2015,0.0,0.0,32765,12000.0,sedan,2,0,1,1.0


In [55]:
def logit_single_factor(factor, data = data_insurance):
    """Finds the accuracy of logit model with onlt one factor.

    Args:
    
        data (DataFrame): the data containg both outcome and explanatory variables.
        
        factor (str): the factor that will predict the response variable.

    Returns:
    
        float : in case that the the factor is not found in the data,
                it willr raise a ValueError. Otherwise, a float that 
                indicates tha ccuracy of the model's prediction, as 
                a measure of the model's performance. 
    """
    if factor is None:
        raise ValueError('The factor cannot be empty')
    elif not factor in data.columns:
        raise ValueError('The factor should be a column of the data')

    try:
        formula = 'outcome ~ ' + factor
        mdl_tmp = logit(formula, data = data).fit()
        mdl_table = mdl_tmp.pred_table()
        accuracy = (mdl_table[0,0] + mdl_table[1,1])/(mdl_table.sum().sum())
        return factor, accuracy
        
    except:
        raise ValueError('modeling did not finished!')


In [56]:
# looping to find the factor with the highest accuracy
best_feature = None

for col in data_insurance.columns:
    if col not in ['id', 'outcome']:
        feature = logit_single_factor(data = data_insurance, factor = col)
        print(feature)
        print('-'*30)
        if best_feature is None:
            best_feature = feature
        elif feature[1] > best_feature[1]:
            best_feature = feature

Optimization terminated successfully.
         Current function value: 0.511794
         Iterations 6
('age', 0.7747)
------------------------------
Optimization terminated successfully.
         Current function value: 0.615951
         Iterations 5
('gender', 0.6867)
------------------------------
Optimization terminated successfully.
         Current function value: 0.467092
         Iterations 8
('driving_experience', 0.7771)
------------------------------
Optimization terminated successfully.
         Current function value: 0.603742
         Iterations 5
('education', 0.6867)
------------------------------
Optimization terminated successfully.
         Current function value: 0.531499
         Iterations 6
('income', 0.7425)
------------------------------
Optimization terminated successfully.
         Current function value: 0.567469
         Iterations 6
('credit_score', 0.7066977156797516)
------------------------------
Optimization terminated successfully.
         Current fun

In [57]:
# Printing the best factor    
print(f"the best feature is: {best_feature[0]} with the accuracy of {best_feature[1]}")

# Creating the final DataFrame
best_feature_df = pd.DataFrame({"best_feature":[best_feature[0]] , 
                                "best_accuracy":[best_feature[1]]
                               })

the best feature is: driving_experience with the accuracy of 0.7771
